# Ollama LLM

이 예제는 Ollama를 이용해서 local 환경에서 다양한 오픈 소스 모델을 서빙할 수 있는 예제입니다.
사용자들은 ollama 사이트에서 특정 오픈소스 모델을 검색해서 사용할 수 있습니다.

* Ollama Website
  * https://ollama.com/search


## Ollama 다운로드 및 설치

In [1]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


### Ollama API 서버 실행
다른 프로그램(애플리케이션)이 모델을 사용할 수 있도록 백그라운드에서 API 서버를 실행

아래 터미널을 클릭해서 창을 열고 아래 명령어를 입력해주세요.
```
ollama serve
```

위의 명령어를 실행했을때 로그가 에러없이 올라가면 정상적으로 ollama API 서버가 실행 된것입니다.

### Ollama 실행 및 모델 다운로드
#### Ollama command 명령어

In [35]:
!ollama

Usage:
  ollama [flags]
  ollama [command]

Available Commands:
  serve       Start ollama
  create      Create a model
  show        Show information for a model
  run         Run a model
  stop        Stop a running model
  pull        Pull a model from a registry
  push        Push a model to a registry
  list        List models
  ps          List running models
  cp          Copy a model
  rm          Remove a model
  help        Help about any command

Flags:
  -h, --help      help for ollama
  -v, --version   Show version information

Use "ollama [command] --help" for more information about a command.


#### 다운로드 받은 모델 리스트 확인

In [22]:
!ollama list

NAME         ID              SIZE      MODIFIED       
gemma3:1B    8648f39daa8f    815 MB    21 minutes ago    


#### 모델 다운로드

In [30]:
!ollama pull gemma3:4B

In [31]:
!ollama list

NAME         ID              SIZE      MODIFIED       
gemma3:4B    a2af6cc3eb7f    3.3 GB    26 seconds ago    
gemma3:1B    8648f39daa8f    815 MB    28 minutes ago    


## Ollama API를 통한 처리

#### Ollama package 설치

In [25]:
%pip install --upgrade --quiet llama-index-llms-ollama

In [27]:
from llama_index.llms.ollama import Ollama

#### Ollama를 통해서 llm 초기화 및 단위테스트

In [33]:
llm = Ollama(
    model="gemma3:4B",
    request_timeout=180.0,
    # Manually set the context window to limit memory usage
    context_window=1024,
)

In [34]:
resp = llm.complete("생성형 AI는 무엇인가요. 간단하게 한줄로 요약해서 알려주세요. ?")
print(resp)

생성형 AI는 기존 데이터를 학습하여 텍스트, 이미지, 오디오 등 새로운 콘텐츠를 스스로 만들어내는 인공지능입니다.


#### Chat 처리

In [ ]:
from llama_index.core.llms import ChatMessage

messages = [
    ChatMessage(
        role="system", content="당신은 과학 기술을 설명해주는 과학자입니다."
    ),
    ChatMessage(role="user", content="태양계에서 가장 온도가 높은 행성은 무엇인가요? 세 문장으로 설명해주세요."),
]
resp = llm.chat(messages)

print(resp)

### Streaming output 처리

#### Streaming 처리

In [37]:
response = llm.stream_complete("토성은 어떤 행성인지 세 문장으로 간단하게 설명해주세요. ")

for r in response:
    print(r.delta, end="")

토성은 태양계에서 가장 큰 행성으로, 아름다운 고리 시스템으로 유명합니다. 

**주요 특징:**

*   **크기:** 지구의 약 9배 크기입니다.
*   **고리:** 토성의 고리는 얼음과 먼지로 이루어진 것으로, 햇빛을 받아 반짝이는 모습이 매우 아름답습니다.
*   **위성:** 토성은 현재까지 88개의 위성을 공식적으로 확인되었으며, 더 많은 위성이 있을 가능성도 있습니다.
*   **대기:** 짙은 황색의 액체 메탄으로 이루어진 대기를 가지고 있으며, 강력한 바람과 폭풍이 자주 발생합니다.
*   **대적점:** 토성의 대기에서 수십 년 동안 지속적으로 발생하는 거대한 폭풍인 '대적점'은 매우 흥미로운 현상입니다.

**기타 정보:**

*   토성은 태양에서 6번째 행성입니다.
*   행성 크기 순서대로 나열하면, 태양계 행성은 수성, 금성, 지구, 목성, 토성, 천왕성, 해왕성 순입니다.

좀 더 자세한 정보를 원하시면 언제든지 질문해주세요.

#### Stream_chat 처리

In [39]:
from llama_index.core.llms import ChatMessage

messages = [
    ChatMessage(
        role="system", content="당신은 과학 기술을 설명해주는 과학자입니다."
    ),
    ChatMessage(role="user", content="해왕성은 어떤 행성인지 세 문장으로 설명해주세요. "),
]
resp = llm.stream_chat(messages)

for r in resp:
    print(r.delta, end="")

물론입니다. 해왕성은 태양계에서 가장 먼 행성으로, 푸른색을 띠는 아름다운 행성입니다. 대기가 매우 희박하고, 엄청난 강풍이 불며, 중심부에는 거대한 고압 기상 현상이 일어나는 것으로 알려져 있습니다. 또한, 해왕성은 태양계에서 가장 추운 행성 중 하나이며, 강력한 폭풍이 지속적으로 발생합니다.

## JSON Mode 출력
Ollama는 모든 응답이 유효한 JSON인지 확인하는 JSON 모드도 지원합니다.
이 기능은 구조화된 출력을 구문 분석해야 하는 도구를 실행할 때 특히 유용합니다.

In [40]:
!ollama pull llama3.1:latest

In [41]:
!ollama list

NAME               ID              SIZE      MODIFIED               
llama3.1:latest    46e0c10c039e    4.9 GB    Less than a second ago    
gemma3:4B          a2af6cc3eb7f    3.3 GB    18 minutes ago            
gemma3:1B          8648f39daa8f    815 MB    46 minutes ago            


In [42]:
llm = Ollama(
    model="llama3.1:latest",
    request_timeout=120.0,
    json_mode=True,
    context_window=1024,
)

In [43]:
response = llm.complete(
    "토성에 대해서 간단하게 태양계에서의 위치, 크기, 구성 물질에 대해서 설명해주세요. 출력은 JSON Object로 출력해주세요. "
)
print(str(response))

{ "name": "토성", "location": "태양계 5번째 행성", "size": "9.45 x 10^4 km", "composition": "암석 및 액체 금속" }


## Structured Outputs with pydantic
LLM에 Pyndatic 클래스를 연결하여 구조화된 출력을 보장할 수도 있습니다. 이렇게 하면 주어진 Pydantic 클래스에 대해 Ollama의 내장 구조화된 출력 기능이 사용됩니다.

In [46]:
from llama_index.core.bridge.pydantic import BaseModel

class Planet(BaseModel):
    """A song with name and artist."""

    name: str
    location: str
    size : str
    composition : str

In [47]:
llm = Ollama(
    model="llama3.1:latest",
    request_timeout=120.0,
    # Manually set the context window to limit memory usage
    context_window=1024,
)

sllm = llm.as_structured_llm(Planet)

In [50]:
from llama_index.core.llms import ChatMessage

query = "토성에 대해서 간단하게 태양계에서의 위치, 크기, 구성 물질에 대해서 설명해주세요. 출력은 JSON Object로 출력해주세요."

response = sllm.chat([ChatMessage(role="user", content=query)])
print(response.message.content)

{"name":"토성","location":"제6 행성","size":"급진적이거나 거대한 것은 아니지만 지구와 비슷한 kích세를 가지고 있다.","composition":"암석과 가스"}


asychronous chat 은 아래와 같이 호출합니다.

In [51]:
response = await sllm.achat(
    [ChatMessage(role="user", content=query)]
)
print(response.message.content)

{"name":"토성","location":"7대 행성 중 5번째","size":"대부분의 행성이 공전하는 반지름보다 9배 더 큰 적외선 및 가스 giant","composition":"수소, 헬륨, 메탄, 아미노메탄, 물 etc."}
